- 모델을 훈련, 저장하는 과정은 GPU 스탠다드에 런타임구성은 표준으로 진행하였다 -> 그 후 예측은 CPU로 진행

- 최종 학습 시간 : 약 1시간
- 버전
  - ultralytics  : 8.0.54
  - cv2          : 4.6.0
  - YOLO         : YOLOv8s

# 패키지 설치

In [1]:
# ultralytics 패키지 설치
!pip install ultralytics                                                                                                  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.9/496.9 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 13.0 MB/s eta 0:00:00


In [4]:
import ultralytics
import cv2

In [5]:
ultralytics.__version__, cv2.__version__

('8.0.54', '4.6.0')

# 모듈 가져오기

In [ ]:
from ultralytics import YOLO
import cv2
import os
import urllib

# 사전학습된 모델 불러오기(yolov8s)

In [ ]:
# load model
model = YOLO("./pretrained/yolov8s.pt")

  0%|          | 0.00/21.5M [00:00<?, ?B/s]

# 데이터 로드

In [ ]:
# 데이터는 roboflow에서 가져옴
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ee7QNGs0EVk0UAyk8epB")
project = rf.workspace("yong331").project("sk-shielders-module-project-3_koreanfood_detecting")
dataset = project.version(3).download("yolov8")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 916.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.6 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=d6f9e40925946550e183a874684d272e1d2d0212decf878c3a3f0b76e7cdc77b
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninsta

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.54, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3 in yolov8:: 100%|██████████| 6032/6032 [00:02<00:00, 2793.80it/s]


# 모델 학습을 위한 준비과정

- data.yaml 파일에서 train, val, test 경로를 지정해야 모델 학습 시에 train.txt, valid.txt, test.txt 파일 내에 저장된 이미지 파일들을 이용하여 학습, 검증, 테스트를 수행 가능
- 따라서, data.yaml 파일에서 train, val, test 경로를 train.txt, valid.txt, test.txt 파일로 지정해줌

In [ ]:
# Roboflow에서 가져온 이미지들 파일에 쓰기
# 밑의 경로는 Roboflow에서 가져온 SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3파일에 이미지를 리스트로 묶기

from glob import glob

train_img_list = glob('/content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/train/images/*.jpg')
valid_img_list = glob('/content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/valid/images/*.jpg')
test_img_list  = glob('/content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/test/images/*.jpg')

In [ ]:
with open('./train.txt', 'w') as f:
	f.write('\n'.join(train_img_list) + '\n')
    
with open('./valid.txt', 'w') as f:
	f.write('\n'.join(valid_img_list) + '\n')
 
with open('./test.txt', 'w') as f:
	f.write('\n'.join(valid_img_list) + '\n')

# 모델 학습

In [ ]:
model.train(data='/content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/data.yaml',epochs=50,imgsz=640)

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=./pretrained/yolov8s.pt, data=/content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.Conv                  [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.C2f                   [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.Conv                  [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.C2f                   [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias
train: Scanning /content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/train/labels... 2108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2108/2108 [00:00<00:00, 2216.01it/s]
train: WARNING ⚠️ /content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/train/images/04_043_04013014_161096464213617_0_jpg.rf.52adba854abc66f6bf9b55beff45db73.jpg: 1 duplicate labels removed
train: New cache created: /content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /content/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/valid/labels... 583 images, 0 backgrounds, 0 corrupt: 100%|██████████| 583/583 [00:00<00:00, 2256.00it/s]
val: New cache created: /content/SK-Shiel

- 위의 학습을 돌리고 나면 runs/detect/train 파일에 weights파일에 best.pt가 생기는데 그것이 모델이다!

# 학습된 모델 불러오기

In [ ]:
# load model
model = YOLO('/content/drive/MyDrive/cloud_ai/모듈_프로젝트_3/이미지분류모델/epoch_50_3.19.pt')

# 테스트

In [ ]:
results = model.predict(source='/content/drive/MyDrive/cloud_ai/모듈_프로젝트_3/한식 이미지/파전.jpg', save=False)
# 테스트 할 때 마다 저장되는 것을 막기위해 save=False로 설정


image 1/1 /content/drive/MyDrive/cloud_ai/모듈_프로젝트_3/한식 이미지/파전.jpg: 448x640 1 pajeon, 544.1ms
Speed: 1.1ms preprocess, 544.1ms inference, 21.3ms postprocess per image at shape (1, 3, 640, 640)




```
성공 : 만두 삼겹살 된장찌개 감자탕 양념치킨 불고기 고등어구이 라면 짬뽕 제육볶음 비빔밥 김밥 짜장면 삼계탕 칼국수 파전
실패 : 김치찌개 순대국밥 떡국
애매 : 떡볶이(김치찌개랑 떡볶이로 인식)
```



In [ ]:
# results가 어떻게 생겼는지 확인
# tensor([[ 51.63605,   6.53531, 296.91885, 215.32338,   0.83917,  14.00000]]) -> 이 부분이 중요
# 앞의 숫자 4개는 bounding box의 좌표이고, 5번째는 확률, 6번째가 클래스 넘버이다 
results

[{ '_keys': ('boxes', 'masks', 'probs'),
   'boxes': ultralytics.yolo.engine.results.Boxes
 type:  torch.Tensor
 shape: torch.Size([1, 6])
 dtype: torch.float32
 tensor([[ 51.63605,   6.53531, 296.91885, 215.32338,   0.83917,  14.00000]]),
   'masks': None,
   'names': { 0: 'Bibimbap',
              1: 'Bulgogi',
              2: 'Godeungeogui',
              3: 'Jjambbong',
              4: 'Ramyun',
              5: 'Yangnyumchicken',
              6: 'duinjangzzigae',
              7: 'gamjatang',
              8: 'gimbap',
              9: 'jeyukbokum',
              10: 'jjajangmyeon',
              11: 'kalguksu',
              12: 'kimchizzigae',
              13: 'mandu',
              14: 'pajeon',
              15: 'samgyetang',
              16: 'samgyubsal',
              17: 'sundaegukbap',
              18: 'tteokbokki',
              19: 'tteokguk'},
   'orig_img': array([[[ 62,  75,  89],
         [ 64,  77,  91],
         [ 71,  84,  98],
         ...,
         [ 30,  

------
# 로컬로 수행(VS Code)

- 모델 저장후 다음 스텝은 실시간 카메라 인식이 있으므로 로컬에서 수행
- 코드만 붙여놈!

In [ ]:
from ultralytics import YOLO
import cv2
import os
import urllib

model = YOLO("C:/Users/USER/Downloads/epoch_50_3.19.pt")  # 본인의 모델저장 path입력

labels =  { 0: 'Bibimbap',
            1: 'Bulgogi',
            2: 'Godeungeogui',
            3: 'Jjambbong',
            4: 'Ramyun',
            5: 'Yangnyumchicken',
            6: 'duinjangzzigae',
            7: 'gamjatang',
            8: 'gimbap',
            9: 'jeyukbokum',
            10: 'jjajangmyeon',
            11: 'kalguksu',
            12: 'kimchizzigae',
            13: 'mandu',
            14: 'pajeon',
            15: 'samgyetang',
            16: 'samgyubsal',
            17: 'sundaegukbap',
            18: 'tteokbokki',
            19: 'tteokguk'}


results = model.predict(source=0, show=True, save=False)

# results를 문자열로 바꿔서 필요한 부분만 받기
str_results = str(results)
li_result = str_results.split('tensor')[-1].split('masks')[0].split(')')[0].split('[')[2].split(']')[0].split(',')
result = int(list(map(float, li_result))[-1])

# 예측한 음식 이름
food_name = labels[result]
# 확률
prob = list(map(float, li_result))[-2]

output = f'This food is {food_name} and the probability is {prob}!!'

